In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

from generators import BlockDiagonalMatrixGenerator, BlockMatrixGenerator
from datasets import MovieLensGenreCastUserData
from models.cmf_vincent.anewton import *
from models.cmf_vincent.utils import *
from models.cmf_vincent.cmf import *

import numpy as np
import time
import logging
import scipy.sparse

In [ ]:
class Args():
    def __init__(self, k, iter, user=0, item=0):
        self.train = "../models/cmf_vincent/data/ml-1m/train.txt"
        self.test = "../models/cmf_vincent/data/ml-1m/test.txt"
        self.item = "../models/cmf_vincent/data/ml-1m/item.txt" if item > 0 else ""
        self.user = "../models/cmf_vincent/data/ml-1m/user.txt" if user > 0 else ""
        self.out = "../output/out-cmf.txt"
        self.link = 'log_dense'
        self.alphas = '0.4'
        if user > 0:
            self.alphas += '-' + str(user)
        if item > 0:
            self.alphas += '-' + str(item)
        self.k = k
        self.reg = 0.1
        self.lr = 0.1
        self.iter = iter
        self.tol = 0.0
        self.verbose = True
        self.boolean = True
        
args = Args(k=15, iter=150, user=0, item=0.2)

In [ ]:
[Xs_trn, Xs_tst, rc_schema, modes] = read_triple_data(args.train, args.test, args.user, args.item, args.link, args.boolean)

display(Xs_trn, Xs_tst)

In [ ]:
data = MovieLensGenreCastUserData(size='100k')
data.load()
data.Xs

In [ ]:
X, W, Y, Z = data.Xs

show_matrix([(X.toarray(), [0, 0], 'X')])

In [ ]:
[Xs_trn, Xs_tst, rc_schema, modes] = read_triple_data(args.train, args.test, args.user, args.item, args.link, args.boolean)

Xs_tst[0] = X

Xs_trn[0] = X
Xs_trn[1] = Y.T

if(args.verbose == 1):
    logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
else:
    logging.basicConfig(level=logging.WARNING, format='[%(levelname)s] %(message)s')

logger = logging.getLogger()
[S, Ns] = get_config(Xs_trn, rc_schema)
alphas = string2list(args.alphas, len(modes))

logger.info('------------------- CMF -------------------')
logger.info('Data: Number of instnace for each entity = {}'.format(list(Ns)))
logger.info('Data: Training size = {}. Testing size = {}'.format(Xs_trn[0].size, Xs_tst[0].size))
logger.info('Settings: k = {}. reg = {}. lr = {}. alpha = {}. modes = {}.'.format(args.k, args.reg, args.lr, alphas, modes))

run_cmf(Xs_trn, Xs_tst, rc_schema, modes, alphas, args, logger)

In [ ]:
rc_schema

In [ ]:
# replace by netflix
from utils import sample

cache_path = "D:/OneDrive - Singapore Management University/cache/netflix_data_small.pickle"
X, factor_info = read_cache(cache_path)

idx_u, _, X = sample(X, axis=0, n_samples=1000, seed=3080)
display(X.shape)

idx_v, _, X = sample(X, axis=1, n_samples=500, seed=4060)
display(X.shape)

display(X)

from datasets import RatioSplit, NoSplit

# X_split = RatioSplit(X=X, test_size=0.2, seed=1000)

X_split = NoSplit(X)

In [ ]:
from datasets import NetflixGenreData

nfg_data = NetflixGenreData(small=True)

_, _, Z = sample(nfg_data.X, idx=idx_v, axis=1)

display(Z)

In [ ]:
import numpy as np

[Xs_trn, Xs_tst, rc_schema, modes] = read_triple_data(args.train, args.test, args.user, args.item, args.link, args.boolean)

Xs_tst[0] = X_split.X_test

Xs_trn[0] = X_split.X_train
Xs_trn[1] = Z.T

if(args.verbose == 1):
    logging.basicConfig(level=logging.INFO, format='[%(levelname)s] %(message)s')
else:
    logging.basicConfig(level=logging.WARNING, format='[%(levelname)s] %(message)s')

logger = logging.getLogger()
[S, Ns] = get_config(Xs_trn, rc_schema)
alphas = string2list(args.alphas, len(modes))

logger.info('------------------- CMF -------------------')
logger.info('Data: Number of instnace for each entity = {}'.format(list(Ns)))
logger.info('Data: Training size = {}. Testing size = {}'.format(Xs_trn[0].size, Xs_tst[0].size))
logger.info('Settings: k = {}. reg = {}. lr = {}. alpha = {}. modes = {}.'.format(args.k, args.reg, args.lr, alphas, modes))

run_cmf(Xs_trn, Xs_tst, rc_schema, modes, alphas, args, logger)